# Pre-processing
- Deleting punctuations except ! - bcos can show emotion
- Removing numbers - no use
- Remove empty rows
- Remove emoji - chatbot won't allow emoji
- Remove more than one space

In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import re
import contractions

In [10]:
csv_file_path = 'Suicide_Detection.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# Display the DataFrame
df.drop(df.columns[0], axis=1, inplace=True)
df = df.head(30000)
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    30000 non-null  object
 1   class   30000 non-null  object
dtypes: object(2)
memory usage: 468.9+ KB
None


In [11]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [12]:
def pre_processing(data):
    data = data.apply(lambda x: ''.join([char for char in x if not char.isdigit()]))
    data = data.apply(lambda x: " ".join(x.split()))
    data = data.apply(lambda x: re.sub(r'[^\w\s!]', '', x))
    data = data.apply(remove_emoji)

    return data

df['text'] = pre_processing(df['text'])

print(df.head())



                                                text        class
0  Ex Wife Threatening SuicideRecently I left my ...      suicide
1  Am I weird I dont get affected by compliments ...  non-suicide
2  Finally is almost over So I can never hear  ha...  non-suicide
3          i need helpjust help me im crying so hard      suicide
4  Im so lostHello my name is Adam  and Ive been ...      suicide


In [13]:
#Map output to be binary 1 or 0
mapping = {'suicide': 1, 'non-suicide': 0}
df['class'] = df['class'].map(mapping)
print(df.head())

                                                text  class
0  Ex Wife Threatening SuicideRecently I left my ...      1
1  Am I weird I dont get affected by compliments ...      0
2  Finally is almost over So I can never hear  ha...      0
3          i need helpjust help me im crying so hard      1
4  Im so lostHello my name is Adam  and Ive been ...      1


# Transformer Model - classification

In [14]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


In [15]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions


In [16]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

vocab_size = 20000  # Only consider the top 20k words
maxlen = 50

X = df['text'].values
y = df['class'].values
y=np.array(y)


# # Tokenize the entire dataset
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(X)

# Convert text to sequences of integers
sequences = tokenizer.texts_to_sequences(X)

# Pad sequences to a fixed length
X = pad_sequences(sequences, maxlen=maxlen, padding="post")

print(X)
print(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42)


[[    2    29     4 ...    10   171   402]
 [   49     2   451 ...     0     0     0]
 [  289    15   275 ...     0     0     0]
 ...
 [  393  1059  9316 ...    97    34   108]
 [   18 10053     9 ...    15   352     9]
 [  800     1    65 ...     0     0     0]]
[1 0 0 ... 0 1 0]


In [17]:
embed_dim = 16  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 16  # Hidden layer size in feed forward network inside transformer


inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.35)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(2, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)


In [18]:
X_train = np.array(X_train)
X_val = np.array(X_val)
X_test = np.array(X_test)

print(model.summary())

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(
    X_train, y_train, batch_size=32, epochs=20, validation_data=(X_val, y_val)
)

test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 50)]              0         
                                                                 
 token_and_position_embeddi  (None, 50, 16)            320800    
 ng (TokenAndPositionEmbedd                                      
 ing)                                                            
                                                                 
 transformer_block (Transfo  (None, 50, 16)            2768      
 rmerBlock)                                                      
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dropout_2 (Dropout)         (None, 16)                0     

In [20]:

text = "cheeseburger sucks"
text = pre_processing(pd.Series(text))

sequences = tokenizer.texts_to_sequences(text)

text = pad_sequences(sequences, maxlen=maxlen, padding="post")
predictions = model.predict(text)
print(predictions)
predicted_class = "non-suicidal" if predictions[0][0] > 0.65 else "suicidal"
print(f"Predicted Class: {predicted_class}")

1/1 [==============================] - 0s 166ms/step
[[0.36016306 0.63983697]]
Predicted Class: suicidal
